In [2]:
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torch.autograd import Variable

from torchvision import transforms, models as torchmodels, datasets

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch.backends.cudnn as cudnn
cudnn.benchmark = True

In [3]:
def load_split_train_test(datadir, valid_size=.2, batch_size=32, num_workers=6, size=(200,200)):
    'Standard read data in function'
    normalize = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    train_transforms = transforms.Compose([transforms.Resize(size),
                                           transforms.ToTensor(),
                                           normalize,
                                       ])
    test_transforms = transforms.Compose([transforms.Resize(size),
                                          transforms.ToTensor(),
                                          normalize,
                                      ])
    train_data = datasets.ImageFolder(datadir,
                    transform=train_transforms)
    test_data = datasets.ImageFolder(datadir,
                    transform=test_transforms)
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    from torch.utils.data.sampler import SubsetRandomSampler
    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    train = torch.utils.data.DataLoader(train_data,
                   sampler=train_sampler, batch_size=batch_size, 
                                        num_workers=num_workers, pin_memory=True)
    test = torch.utils.data.DataLoader(test_data,
                   sampler=test_sampler, batch_size=batch_size, 
                                       num_workers=num_workers, pin_memory=True)
    return train, test

In [7]:
# Setup variables
batch_size = 128
epochs = 10
learning_rate = 1e-3
imgsize = (128, 128)
transfer = False
root = 'data/'
train_loader, test_loader = load_split_train_test(root, batch_size=batch_size, size=imgsize)

FileNotFoundError: [Errno 2] No such file or directory: 'data/'

In [ ]:
def init_weights(m):
    'Kaiming weights'
    if type(m) == nn.Conv2d:
        torch.nn.init.kaiming_normal_(m.weight)

In [4]:
class Hierarchical:
    'A hierarchical classifier'
    def __init__(self, num_classes, epochs, batch_size, learning_rate):
        'Initialize parameters, and other meme stuff'
        self.resnets = []
        for i in range(num_classes):
            self.resnets.append(torchmodels.resnet18(num_classes=2))
            self.resnets[i].apply(init_weights)
            #if torch.cuda.is_available():
            #    self.resnets[i] = self.resnets[i].cuda()
                
        # Optimizers and criterion
        self.optimizers = []
        for i in range(num_classes):
            self.optimizers.append(optim.Adam(self.resnets[i].parameters(), lr=learning_rate))
        
        self.criterion = nn.BCELoss()
        if torch.cuda.is_available():
            self.criterion = self.criterion.cuda()
    
    def train_idx(self, idx, epoch, train_loader):
        'Trains a net for 1 epoch'
        if torch.cuda.is_available():
            self.resnets[idx] = self.resnets[idx].cuda()
        self.resnets[idx].train()
        for batch_idx, (features, labels) in enumerate(train_loader):
            features, labels = Variable(features), Variable(labels)
            if torch.cuda.is_available():
                features, labels = features.cuda(), labels.cuda()
            optimizer.zero_grad()
            output = self.resnets[idx](features)
            loss = 
        self.resnets[idx].cpu()
            
            